# Discogs alignment

List of concert halls: https://www.discogs.com/lists/concert-halls-and-other-venues_0-9/141428
List of studios: https://www.discogs.com/help/forums/topic/277190
List of churches: https://www.discogs.com/lists/Churches/261273?page=123
discogs API: https://www.discogs.com/developers/

example: 
https://www.discogs.com/label/280422-Jesus-Christus-Kirche-Berlin

## XML dump

https://data.discogs.com/?prefix=data/2018/

In [1]:
import xml.etree.cElementTree as ET # better for files > 1Gb
#import lxml.etree as ET

parser = ET.iterparse('./discogs_20180401_labels.xml')

for _, item in parser:
    if item.tag != 'label':
        continue
    # print()
    # print(item.text)
    # print(item.items())
    # print(item.getchildren())
    # if item.find('id') is not None:
    #     print(item.find('id').text)
    # if item.find('name') is not None:
    #     print(item.find('name').text)
    if item.find('name') is not None and 'Jesus-Christus' in item.find('name').text:
        place = item
        print(item.find('name').text)
        print(item.find('id').text)
        print(item.find('profile').text)
        print(item.find('contactinfo').text)
        print(item.find('data_quality').text)
        print(item.find('urls').find('url').text)

Jesus-Christus-Kirche, Berlin
280422
Also refered to as Jesus-Christus-Kirche, Jesus Christuskirche Berlin, Berlin-Dahlem or Jesus-Christus-Kirche, Dahlem.
Hittorfstraße 23
14195 Berlin
Germany

Disponent Horst Langheinrich: klangh@gmx.de
Tel.: 0171 60 53 248 
Tel.: 831 45 08
Fax: 83 22 94 93

Correct
http://de.wikipedia.org/wiki/Jesus-Christus-Kirche_Dahlem


## XML to JSON

In [ ]:
# discogs-xml2db
!./discogsparser.py ./discogs_201804018_artists.xml -n 1000 -o json > artists.json

In [1]:
import json
import requests

In [26]:
def parse(artist, url):
    wiki = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&format=json&titles={}'
    resp = requests.get(wiki.format(url.split('/')[-1]))
    claims = list(resp.json()['entities'].values())[0].get('claims')
    if not claims:
        return
    for mblink in claims.get('P434', []):
        mbid = mblink['mainsnak']['datavalue']['value']
        mb_url = 'https://musicbrainz.org/ws/2/artist/{}?fmt=json&inc=url-rels'
        resp = requests.get(mb_url.format(mbid))
        mbrels = resp.json().get('relations', [])
        mbrels = filter(lambda rel: rel['type'] == 'discogs', mbrels)
        if mbrels:
            print("Two-ways relation found", artist['id'], mbid)
        else:
            print("One-way relation found", artist['id'], mbid)

In [31]:
with open('../discogs/artists.json') as fd:
    for line in fd:
        if line.startswith('Parsed'):
            continue
        artist = json.loads(line.strip())
        for url in artist['urls']:
            if 'wikipedia' in url:
                parse(artist, url)

Two-ways relation found 1046 3fff189a-1113-4006-b5a1-31382bd16bc2
Two-ways relation found 1049 092ca127-2e07-4cbd-9cba-e412b4ddddd9
Two-ways relation found 1060 2b340ddd-1c54-407d-a62d-7b14b294378b
Two-ways relation found 1060 2b340ddd-1c54-407d-a62d-7b14b294378b
Two-ways relation found 1091 d430311e-c619-4a29-bacf-049ef02f2998
Two-ways relation found 1094 17d69e72-9cf2-496f-aa64-ba238de2087e
Two-ways relation found 1094 a703b91d-aadc-4021-8b61-05cfc134614f
Two-ways relation found 1096 3d05eb8b-1644-4143-9a61-b28e33c4d85f
Two-ways relation found 1100 09039d5e-335c-4c08-9474-a83d1b838f23
Two-ways relation found 1102 e569aee8-33ff-48e5-b0ed-4ceba1a37714
Two-ways relation found 1102 a110acfe-212d-44b2-b729-18f0f94ce47e
Two-ways relation found 1102 57dd6359-f4ef-422a-9566-b8f54a0904fe


https://api.discogs.com/artists/95544